# Decision Tree Example implementation without using libraries 

Here we develop an example dataset and write functions for implementing decision tree using CART algorithm (Classification and Regression Trees)  based on Gini's impurity

*Based on implementation of Josh Gordon
https://github.com/random-forests/tutorials/blob/master/decision_tree.py 

> Indented block




In [82]:
from __future__ import print_function

In [110]:
training_data = [
                 [210, 160, 'Smoker','Cancer'],
                 [200, 180, 'Smoker','Cardiovascular' ],
                 [210, 190, 'Smoker','Cardiovascular'],
                 [210, 150, 'Smoker','Cancer'],
                 [110, 140, 'nonesmoker', 'Normal'],
                 [110, 120, 'nonesmoker', 'Normal'],
                 [140, 120, 'nonesmoker', 'Normal']
]

In [111]:
features = ['Chol', 'SPB', 'SmokerOrNot']

In [125]:
testing_data = [
                 [190, 160, 'Smoker', 'Cancer'],
                 [185, 180, 'Smoker', 'Cardiovascular'],
                 [180, 175, 'Smoker', 'Cardiovascular'],
                 [120, 150, 'nonesmoker', 'Normal'],
                 [100, 110, 'nonesmoker', 'Normal'],
                 [130, 110, 'nonesmoker', 'Normal']
]

In [86]:
def unique_values(rows, col):
  return set([row[col] for row in rows])

In [87]:
unique_values(training_data, 0)

{100, 120, 130, 180, 185, 190}

In [88]:
unique_values(training_data, 1)

{110, 150, 165, 170}

In [89]:
unique_values(training_data, 2)

{'Smoker', 'nonesmoker'}

In [90]:
def class_counts(rows):
  counts = {}

  for row in rows:
    # our label is the last column
    label = row[-1]
    if label not in counts:
      counts[label] = 0
    counts[label] +=1
  
  return counts  

In [91]:
class_counts(training_data)

{'Normal': 3, 'Sick': 3}

In [92]:
class Question:
  def __init__(self, column, value):
    self.column = column
    self.value = value 

  def match(self, example):
    val = example[self.column]
    
    if type(self.value) == int or float:
      return val >= self.value
    else:
      return val == self.value
  
  def __repr__(self):

    condition = "=="

    if type(self.value) == int or float:
      condition = ">="
    return "Is %s %s %s?" % (features[self.column], condition, str(self.value))


In [93]:
Question(0, 3)

Is Chol >= 3?

In [94]:
def gini(rows):
  """Gini Impurity for a list of data rows """
  counts = class_counts(rows)
  impurity =1 

  for label in counts:
    prob_of_label = counts[label] / float(len(rows))
    impurity -= prob_of_label**2
  return impurity


In [95]:
no_mixiing = [['Sick'], ['Sick']]

gini(no_mixiing)

0.0

In [96]:
mixiing = [['Sick'], ['Normal']]
gini(mixiing)

0.5

In [99]:
more_mixiing = [['normal'], ['flu'], ['Cardiovascular'], ['Cancer'], ['COVID-19']]
gini(more_mixiing)

0.7999999999999998

In [100]:
def info_gain(left, right, current_uncertainty):
  p = float(len(left)/(len(left) + len(right)))
  return current_uncertainty - p* gini(left) - (1-p)*gini(right)


In [113]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6111111111111112

In [119]:
def partition(rows, question):
    """Partitions a dataset.
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


In [120]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [121]:
class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)


class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


def build_tree(rows):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [122]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
# my_tree = build_tree(training_data)
# classify(training_data[0], my_tree)
#######

def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [123]:
my_tree = build_tree(training_data)

print_tree(my_tree)

Is SmokerOrNot >= nonesmoker?
--> True:
  Predict {'Normal': 3}
--> False:
  Is SPB >= 175?
  --> True:
    Predict {'Cardiovascular': 2}
  --> False:
    Predict {'Cancer': 1}


In [126]:
testing_data = [
                 [190, 160, 'Smoker', 'Cancer'],
                 [185, 180, 'Smoker', 'Cardiovascular'],
                 [180, 175, 'Smoker', 'Cardiovascular'],
                 [120, 150, 'nonesmoker', 'Normal'],
                 [100, 110, 'nonesmoker', 'Normal'],
                 [130, 110, 'nonesmoker', 'Normal']
]

for row in testing_data:
        print ("Actual: %s. Predicted: %s" %
               (row[-1], print_leaf(classify(row, my_tree))))

Actual: Cancer. Predicted: {'Cancer': '100%'}
Actual: Cardiovascular. Predicted: {'Cardiovascular': '100%'}
Actual: Cardiovascular. Predicted: {'Cardiovascular': '100%'}
Actual: Normal. Predicted: {'Normal': '100%'}
Actual: Normal. Predicted: {'Normal': '100%'}
Actual: Normal. Predicted: {'Normal': '100%'}
